In [134]:
import pandas as pd
import numpy as np

pred_df = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/df_prepro_2.csv", index_col=0)

In [135]:
# Create dummy variables for 'Home' and 'Away' columns in training data
pred_df = pd.get_dummies(pred_df, columns=['Home', 'Away'])

In [136]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

X = pred_df.drop([ 'FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)
Y = pred_df['FTR']

X_test = pred_df.drop(['FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)
y_test = pred_df['FTR']

/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_3020/3654059462.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = pred_df.drop([ 'FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)
/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_3020/3654059462.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = pred_df.drop(['FTR', 'Date', 'B365_guess', 'B365_right_or_wrong', ], 1)


In [36]:
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [137]:
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)
            
Y = transform_results(Y)
y_test = transform_results(y_test)

In [138]:
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test_scaled = scaler.fit_transform(X_test)

In [139]:
# Split our data into train/validation. We are using the 
# most recent full season (18/19) for validation data.
X_train_scaled = X[:-160]
y_train = Y[:-160]
X_test_scaled = X[-160:]
y_test = y_test[-160:]

In [140]:
print('Number of matches in training data:', X_train_scaled.shape[0])
print('Number of matches in test data:', X_test_scaled.shape[0])
print('Number of features:', X_train_scaled.shape[1])

Number of matches in training data: 1700
Number of matches in test data: 160
Number of features: 85


In [188]:
# input dimension is number of features
input_dim = X_train_scaled.shape[1]

activation_func = 'relu'
kernel_init = 'glorot_normal'
learning_rate = 0.0001
batch_size = 16

model = keras.Sequential([
    Dense(48, input_shape=(input_dim,), activation=activation_func),
    Dropout(0.5),
    Dense(16),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [189]:
es = EarlyStopping(monitor='loss', patience=3, verbose=1)

In [89]:
from keras.optimizers import Adam, SGD

In [190]:
# Define your learning rate
learning_rate = 0.0001

# Define your optimizer with the specified learning rate
opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compile your model with the optimizer
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [191]:
model.fit(X_train_scaled, y_train, batch_size=batch_size, 
              callbacks=[es], epochs=500, verbose=0)

Epoch 21: early stopping


In [192]:
train_loss, train_acc = model.evaluate(X_train_scaled, y_train)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print('Training loss:', train_loss)
print('Training accuracy:', train_acc)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - accuracy: 0.4927 - loss: 0.9924
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5082 - loss: 1.0420 
Training loss: 0.9877327084541321
Training accuracy: 0.5147058963775635
Test loss: 1.0150901079177856
Test accuracy: 0.5375000238418579


In [193]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_preds = model.predict(X_test_scaled)
y_pred_argmax = [np.argmax(i) for i in y_preds]
print(classification_report(y_test, y_pred_argmax))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.58      0.78      0.66        73
           1       0.30      0.16      0.21        38
           2       0.56      0.47      0.51        49

    accuracy                           0.54       160
   macro avg       0.48      0.47      0.46       160
weighted avg       0.51      0.54      0.51       160



In [194]:
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

y_preds = model.predict(X_test_scaled)

for i in range(len(X_test_scaled)):
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_test[i])
    print('Favourite', np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', X_test['B365H'][i],       
          X_test['B365D'][i], X_test['B365A'][i])
    
    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365H'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
                
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365H'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
            
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365D'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds +=( wager * X_test['B365D'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365A'][i])
        if odds_diff >  min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365A'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
   
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(X_test)} matches.')
print(f'No bet placed {no_bets} times')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.37787855 0.36209962 0.26002178]
Home, Draw and Away odds 5.25 3.8 1.65
Funds 90

Prediction 0
Actual 1
Favourite 2
Prediction proba [0.7532641  0.15136291 0.09537294]
Home, Draw and Away odds 4.75 4.2 1.65
Funds 80

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.51536554 0.27976894 0.2048655 ]
Home, Draw and Away odds 1.66 3.75 5.5
Funds 80

Prediction 0
Actual 0
Favourite 2
Prediction proba [0.39793628 0.23652521 0.36553857]
Home, Draw and Away odds 2.8 3.2 2.6
Funds 98.0

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.713071   0.19357158 0.09335731]
Home, Draw and Away odds 2.0 3.2 4.2
Funds 108.0

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.26319388 0.35177535 0.3850307 ]
Home, Draw and Away odds 1.6 3.8 6.5
Funds 98.0

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.3558515  0.35293722 0.29121128]
Home, Draw and Away odds 2.15 3.2 3.6
Funds 98.0

Prediction 0
A